In [ ]:
!pip install awscli
import awscli
from google.colab import drive
!pip install pydicom
drive.mount("/content/drive", force_remount=True)

In [ ]:
!cat /content/drive/My\ Drive/config/awscli.ini
path = "/content/drive/My Drive/config/awscli.ini"
 
import os
!export AWS_SHARED_CREDENTIALS_FILE=/content/drive/My\ Drive/config/awscli.ini
path = "/content/drive/My Drive/config/awscli.ini"
os.environ['AWS_SHARED_CREDENTIALS_FILE'] = path

!aws s3 cp s3://medical-image-segmentation/lungs/70-10-20/train.zip .
!aws s3 cp s3://medical-image-segmentation/lungs/70-10-20/val.zip .
!aws s3 cp s3://medical-image-segmentation/lungs/70-10-20/test.zip .
!aws s3 cp s3://medical-image-segmentation/lungs/70-10-20/train-output.zip .
!aws s3 cp s3://medical-image-segmentation/lungs/70-10-20/val-output.zip .
!aws s3 cp s3://medical-image-segmentation/lungs/70-10-20/test-output.zip .

!unzip train.zip
!unzip val.zip
!unzip test.zip
!unzip train-output.zip
!unzip val-output.zip
!unzip test-output.zip

In [ ]:
# coding: utf-8
import os
import tensorflow as tf
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from tensorflow import keras
import matplotlib.pyplot as plt
!pip install keras_unet
from keras_unet.models import custom_unet

SEED = 909
BATCH_SIZE_TRAIN = 4
BATCH_SIZE_VAL = 4
BATCH_SIZE_TEST = 4
IMAGE_HEIGHT=256
IMAGE_WIDTH=256
IMG_SIZE = (IMAGE_HEIGHT,IMAGE_WIDTH)
NUM_TRAIN = 6651
NUM_VAL = 932
NUM_TEST = 1950

def create_train(img_path, mask_path, BATCH_SIZE):
    data_gen_args = dict(rescale=1./255)
    img_datagen = ImageDataGenerator(**data_gen_args)
    mask_datagen = ImageDataGenerator(**data_gen_args)
    
    img_generator = img_datagen.flow_from_directory(img_path, target_size=IMG_SIZE, class_mode=None,
       color_mode='grayscale', batch_size=BATCH_SIZE, seed=SEED)

    mask_generator = mask_datagen.flow_from_directory(mask_path, target_size=IMG_SIZE, class_mode=None,
       color_mode='grayscale', batch_size=BATCH_SIZE, seed=SEED)
    return zip(img_generator, mask_generator)

train_img_path = os.path.join('train-output', 'images')
train_mask_path = os.path.join('train-output', 'masks')

val_img_path = os.path.join('val-output', 'images')
val_mask_path = os.path.join('val-output', 'masks')

test_img_path = os.path.join('test-output', 'images')
test_mask_path = os.path.join('test-output', 'masks')

train_generator = create_train(train_img_path, train_mask_path, BATCH_SIZE_TRAIN)
val_generator = create_train(val_img_path, val_mask_path, BATCH_SIZE_TRAIN)
test_generator = create_train(test_img_path, test_mask_path, BATCH_SIZE_TRAIN)

NUM_OF_EPOCHS = 100

/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.6) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


-----------------------------------------
keras-unet init: TF version is >= 2.0.0 - using `tf.keras` instead of `Keras`
-----------------------------------------
Found 6651 images belonging to 1 classes.
Found 6651 images belonging to 1 classes.
Found 932 images belonging to 1 classes.
Found 932 images belonging to 1 classes.
Found 1950 images belonging to 1 classes.
Found 1950 images belonging to 1 classes.


In [ ]:
def display(display_list):
    plt.figure(figsize=(15,15))
    title = ['Input', 'True Mask', 'Predicted Mask']
    for i in range(len(display_list)):
        plt.subplot(1, len(display_list), i+1)
        plt.title(title[i])
        plt.imshow(tf.keras.preprocessing.image.array_to_img(display_list[i]), cmap='gray')
    plt.show()

def show_prediction(datagen, num=1):
    for i in range(0,num):
        image,mask = next(datagen)
        pred_mask = model.predict(image)[0] > 0.5
        display([image[0], mask[0], pred_mask])

In [ ]:
show_prediction(train_generator, 10)

In [ ]:
import keras.backend as K
import math
def DiceLoss(targets, inputs, smooth=1e-6):
    inputs = K.flatten(inputs)
    targets = K.flatten(targets)
    
    intersection = K.sum(targets * inputs)
    dice = (2*intersection + smooth) / (K.sum(targets) + K.sum(inputs) + smooth)
    return 1 - dice

In [ ]:
model = custom_unet(
    input_shape=(256, 256, 1),
    use_batch_norm=True,
    num_classes=1,
    filters=64,
    dropout=0.25,
    output_activation='sigmoid')

In [ ]:
EPOCH_STEP_TRAIN = NUM_TRAIN // BATCH_SIZE_TRAIN
EPOCH_STEP_VAL = NUM_VAL // BATCH_SIZE_VAL
EPOCH_STEP_TEST = NUM_TEST // BATCH_SIZE_TEST
model.compile(optimizer='adam', loss=DiceLoss, metrics=[DiceLoss, tf.keras.metrics.Precision(), tf.keras.metrics.Recall()], run_eagerly=True)

In [ ]:
model.fit_generator(generator=train_generator, 
                    steps_per_epoch=EPOCH_STEP_TRAIN, 
                    validation_data=val_generator, 
                    validation_steps=EPOCH_STEP_,
                    epochs=50)

In [ ]:
model.save(f'drive/MyDrive/UNET3-A99-P93-R88-{IMAGE_HEIGHT}_{IMAGE_WIDTH}.h5')

In [ ]:
model = keras.models.load_model(f'drive/MyDrive/UNET2-A99-P93-R88-{IMAGE_HEIGHT}_{IMAGE_WIDTH}.h5', custom_objects={"DiceLoss": DiceLoss})

In [ ]:
def dice_gen(img_path, mask_path, BATCH_SIZE):
    data_gen_args = dict(rescale=1./255)
    img_datagen = ImageDataGenerator(**data_gen_args)
    mask_datagen = ImageDataGenerator(**data_gen_args)
    
    img_generator = img_datagen.flow_from_directory(img_path, target_size=IMG_SIZE, class_mode=None,
       color_mode='grayscale', batch_size=BATCH_SIZE, seed=SEED, shuffle=False)

    mask_generator = mask_datagen.flow_from_directory(mask_path, target_size=IMG_SIZE, class_mode=None,
       color_mode='grayscale', batch_size=BATCH_SIZE, seed=SEED, shuffle=False)
    return img_generator, mask_generator

In [ ]:
from pydicom import dcmread
import statistics
from os import listdir
from os.path import join

# make sure metrics.py and lookup_tables.py is uploaded into runtime
from metrics import compute_surface_distances, compute_surface_overlap_at_tolerance

def get_distances(path):
  distances = {}
  for subdir in listdir(path):
    for image in listdir(join(path, subdir, "images")):
      dimensions = dcmread(join(path, subdir, "images", image)).PixelSpacing
      png_name = "lung_l/" + subdir + "-" + image + ".png"
      distances[png_name] = [d * 2 for d in dimensions]                         # change this ratio if image dimensions change
  return distances

def get_surface_dice_values(img_path, mask_path, batch_size, iterations, pixel_distances):
  index = 0
  dice_list = []
  image_gen, mask_gen = dice_gen(img_path, mask_path, batch_size)
  files = image_gen.filenames

  for i in range(iterations):
    image, mask = next(image_gen), next(mask_gen)
    pred_masks = model.predict(image)
    

    for j in range(batch_size):
      pred_mask = pred_masks[j] > 0.5                                           # not sure if this is correct thing to do
      bool_mask = mask[j].astype(bool)
      mask_gt = bool_mask.reshape((256, 256))
      predicted_mask = pred_mask.reshape((256, 256))

      surface_distances = compute_surface_distances(mask_gt, predicted_mask, pixel_distances[files[index]])
      surface_dice = compute_surface_overlap_at_tolerance(surface_distances, 1.9)
      
      dice_list.extend(surface_dice)
      index += 1
  return dice_list
  
def compute_surface_dice_stats(dice_list):
  filtered_dice_list = []
  for val in dice_list:
    if not math.isnan(val):
      filtered_dice_list.append(val)
  filtered_dice_list.sort()
  mean = sum(filtered_dice_list) / len(filtered_dice_list)
  median = filtered_dice_list[len(filtered_dice_list) // 2]
  std_dev = statistics.stdev(filtered_dice_list)
  return mean, median, std_dev


In [ ]:
# split can be either train, val or test

def surface_dice(split):
  if split == "train":
    img_path, mask_path = train_img_path, train_mask_path
    batch_size, iterations = BATCH_SIZE_TRAIN, EPOCH_STEP_TRAIN
  elif split == "val":
    img_path, mask_path = val_img_path, val_mask_path
    batch_size, iterations = BATCH_SIZE_VAL, EPOCH_STEP_VAL
  elif split == "test":
    img_path, mask_path = test_img_path, test_mask_path
    batch_size, iterations = BATCH_SIZE_TEST, EPOCH_STEP_TEST
  else:
    assert False, "invalid split"
  
  pixel_distances = get_distances(split)
  dice_list = get_surface_dice_values(img_path, mask_path, batch_size, iterations, pixel_distances)
  return compute_surface_dice_stats(dice_list)
  

In [ ]:
print(surface_dice("test"))

Found 1950 images belonging to 1 classes.
Found 1950 images belonging to 1 classes.
0


/content/metrics.py:389: RuntimeWarning: invalid value encountered in double_scalars
  np.sum(surfel_areas_gt))
/content/metrics.py:392: RuntimeWarning: invalid value encountered in double_scalars
  np.sum(surfel_areas_pred))


100
200
300
400
(0.5513088331263296, 0.5927090411121572, 0.16541430069340934)


# Code below is for testing purposes

1. reformat images to be 512 x 512 to allow for better tolerances 
2. check surface_dice calcuations and make sure they are correct

In [ ]:
from pydicom import dcmread

def get_distances(path):
  distances = {}
  for subdir in listdir(path):
    for image in listdir(join(path, subdir, "images")):
      dimensions = dcmread(join(path, subdir, "images", image)).PixelSpacing
      png_name = "lung_l/" + subdir + "-" + image + ".png"
      distances[png_name] = [d for d in dimensions]                         # change this ratio if image dimensions change
  return distances

distances = get_distances("train")

In [ ]:
image_gen, mask_gen = dice_gen(train_img_path, train_mask_path, BATCH_SIZE_TRAIN)
files = image_gen.filenames

Found 6651 images belonging to 1 classes.
Found 6651 images belonging to 1 classes.


In [ ]:
index = 0
dice_list = []
for i in range(1662):
  image,mask = next(image_gen), next(mask_gen)
  pred_masks = model.predict(image)
  for j in range(4):
    pred_mask = pred_masks[j] > 0.5
    bool_mask = mask[j].astype(bool)
    
    mask_gt = bool_mask.reshape((256, 256))
    predicted_mask = pred_mask.reshape((256, 256))

    surface_distances = compute_surface_distances(mask_gt, predicted_mask, [1.9, 1.9])
    surface_dice = compute_surface_overlap_at_tolerance(surface_distances, 1.9)
    
    dice_list.extend(surface_dice)
    
    index += 1

In [ ]:
count = 0
total = 0
for val in dice_list:
  if not math.isnan(val):
    count += 1
    total += val

print(total/count)

0.8876650726948787


In [ ]:
print(dice_list)

[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 0.0, nan, nan, nan, nan, nan, nan, nan, 0.0, nan, nan, nan, nan, nan, nan, nan, nan, 1.0, 1.0, 0.8599316841507596, 0.933058261758408, 0.9900083191854215, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9865510666395846, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9952340867985054, 1.0, 1.0, 1.0, 0.9900283465436729, 0.8730473937305941, 0.975263693278846, 0.9800271163342484, 0.9965604424051601, 1.0, 0.9746552121178854, 0.10355339059327374, 0.9782623416471518, 0.9807922969454151, 0.9946255634684573, 1.0, 1.0, 1.0, 0.9973554210986032, 1.0, 0.9948580891939856, 1.0, 0.9974636938553405, 1.0, 0.9923656441447171, 0.9948250234350134, 1.0, 1.0, 0.9975764468385668, 0.9975102500080574, 0.995210588948879, 1.0, 0.993371021890863, 1.0, 0.980145940192369, 0.9977269163916047, 0.97510778799494, 0.99560344466308, 0.9760730242919254, 0.9902608607588155, 0.982057300164694